In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Visualization
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
import plotly
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.io as pio

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### In this competition you will work with a challenging time-series dataset consisting of daily sales data, kindly provided by one of the largest **Russian software firms - 1C Company.**

Submissions are evaluated by **root mean squared error (RMSE)**. True target values are clipped into [0,20] range.

> **1. Preprocessing**

In [2]:
# Read items data
items = pd.read_csv("/kaggle/input/competitive-data-science-predict-future-sales/items.csv")

# Read Item Categories data
item_category = pd.read_csv("/kaggle/input/competitive-data-science-predict-future-sales/item_categories.csv")

# Read sales data
train = pd.read_csv("/kaggle/input/competitive-data-science-predict-future-sales/sales_train.csv")

# Read shop inforations data
shops = pd.read_csv("/kaggle/input/competitive-data-science-predict-future-sales/shops.csv")

# Read test data
test = pd.read_csv("/kaggle/input/competitive-data-science-predict-future-sales/test.csv")

**File descriptions**  
- sales_train.csv - the training set. Daily historical data from January 2013 to October 2015.  
- test.csv - the test set. You need to forecast the sales for these shops and products for November 2015.   
- sample_submission.csv - a sample submission file in the correct format.    
- items.csv - supplemental information about the items/products.    
- item_categories.csv  - supplemental information about the items categories.   
- shops.csv- supplemental information about the shops.   

**Data fields**
* ID - an Id that represents a (Shop, Item) tuple within the test set   
* shop_id - unique identifier of a shop    
* item_id - unique identifier of a product   
* item_category_id - unique identifier of item category   
* item_cnt_day - number of products sold. You are predicting a monthly amount of this measure  
* item_price - current price of an item  
* date - date in format dd/mm/yyyy  
* date_block_num - a consecutive month number, used for convenience. January 2013 is 0, February 2013 is 1,..., October 2015 is 33  
* item_name - name of item  
* shop_name - name of shop  
* item_category_name - name of item category  

In [3]:
items.head(10)

In [4]:
item_info = pd.merge(items, item_category, how="left", on="item_category_id")
item_info.isnull().sum()
print("{0} rows".format(item_info.shape[0]))

In [5]:
print("{0} rows".format(train.shape[0]))
train.head()

In [6]:
# Convert "date" to datetime datatype
train["date"] = pd.to_datetime(train["date"], format="%d.%m.%Y")
print("New 'date' datatype is:", train["date"].dtype)

In [7]:
shop_sales = pd.merge(train, shops, how="left", on="shop_id")
shop_sales.head()

In [8]:
print("Sales data shape:", train.shape)
print("After merged:", shop_sales.shape)

Merge item_info with shop_sales to produce a sales data

In [9]:
df = pd.merge(item_info, shop_sales, how="right", on="item_id")
origin_rows = df.shape[0]
print("After merged: {0} records".format(origin_rows))

In [10]:
to_drop = ["item_name", "shop_name", "item_category_name"]
df = df.drop(columns=to_drop)
df.head()

In [11]:
df.dtypes

In [12]:
df["item_cnt_day"] = df["item_cnt_day"].astype('int64')
df["item_category_id"] = df["item_category_id"].astype("category")
df["item_id"] = df["item_id"].astype("category")
df["shop_id"] = df["shop_id"].astype("category")

Some records have **negative** number of item sold. We should convert them to positive value.

In [13]:
neg = df[df["item_cnt_day"]<0].shape[0]
print("Number of negative values:",neg)

In [14]:
# Convert to positive number
df["item_cnt_day"] = df["item_cnt_day"].apply(lambda x: x*(-1) if x<0 else x)
print("Number of negative :", df[df["item_cnt_day"]<0].shape[0])

In [15]:
print("Number of item price negative or equal zero:", df[df["item_price"]<=0].shape[0])
df = df[df["item_price"]>0]
print("Number of item price negative or equal zero after dropping:", df[df["item_price"]<=0].shape[0])

**Check for null values**

In [16]:
df.isna().sum()

**Many outliers**  
Drop rows if item_price < 0.01 quantile and > 0.99 quantile

In [17]:
df = df[(df["item_price"]<=df["item_price"].quantile(.95))&(df["item_price"]>=df["item_price"].quantile(.05))]
df = df[(df["item_cnt_day"]<=df["item_cnt_day"].quantile(.95))&(df["item_cnt_day"]>=df["item_cnt_day"].quantile(.05))]
print("After merged: {0} records".format(origin_rows))
print("After drop outliers: {0} records,\nWe dropped {1} rows".format(df.shape[0], origin_rows-df.shape[0]))

> **2. Exploratory Analysis**

Qs

In [18]:
df.describe().round(2)

In [19]:
num_vars = ["item_price", "date_block_num"]
fig, ax = plt.subplots(figsize=(14,5), dpi=200, nrows=1, ncols=2)

for num, axis in zip(num_vars, ax.ravel()):
    sns.histplot(data=df,
                 x=num, ax=axis,
                 bins=30, kde=True,
                 kde_kws=dict(bw_adjust=3)
                )
    axis.set_ylabel('')

plt.suptitle("Distribution of numeric values")

**Total quantity sold over time**

In [20]:
total = df.set_index("date").groupby(by=[pd.Grouper(freq="M")])["item_cnt_day"].sum().reset_index()

fig = px.line(data_frame=total,
              x="date", y="item_cnt_day",
              hover_name=sorted(df["date_block_num"].unique()),
              labels = {"item_cnt_day":"Total quantity sold"}
             )

fig.update_layout(legend=dict(title='Categories'),
                  plot_bgcolor='#fafafa',
                  title="Total item sold by months (Jan 2013-Oct 2015)"
                 )

fig.update_xaxes(title="Date")
fig.update_yaxes(title="Total quantity sold")

In [21]:
sale_category = df.groupby(by=["item_category_id", "date_block_num"])["item_cnt_day"].sum().reset_index()

fig = px.bar(sale_category,
             x="item_category_id",
             y="item_cnt_day",
             animation_frame = "date_block_num"
            )

fig.update_xaxes(title="Category Id")
fig.update_yaxes(title="Total quantity sold")
fig.update_layout(width=1000, height=600,
                  title="Total quantity sold by category"
                 )

Which shops had the highest number of sold?

In [22]:
sale_shop = df.groupby(by=["shop_id", "date_block_num"])["item_cnt_day"].sum().reset_index()

fig = px.bar(sale_shop,
             x="shop_id",
             y="item_cnt_day",
             animation_frame="date_block_num"
            )

fig.update_xaxes(title="Shop Id")
fig.update_layout(width=1000, height=600,
                  legend=dict(title="Top 5 category"),
                  title="Total item sold by Shop_id"
                 )

> **3. Time-series forcasting with S-ARIMA**

In [23]:
!pip install statsmodels --upgrade
import statsmodels
from statsmodels.graphics.tsaplots import plot_pacf
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsmodels.tsa.stattools import adfuller

In [ ]:
ad_fuller_result = adfuller(df["item_cnt_day"])
print(f'ADF Statistic: {ad_fuller_result[0]}')
print(f'p-value: {ad_fuller_result[1]}')